In [1]:
import sys
sys.path.insert(0, '/global/homes/d/dbrookes/design_icml/')
import itertools
from keras.layers import Input, Dense, Reshape, Flatten
from keras import layers, initializers
from keras.models import Model, load_model
import keras.backend as K
import tensorflow as tf
import numpy as np
from seqtools import SequenceTools as ST
from gfp_gp import SequenceGP
from util import AA, AA_IDX
from util import build_vae
from sklearn.model_selection import train_test_split, ShuffleSplit
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import pandas as pd
from gan import WGAN
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
import scipy.stats
from scipy.stats import norm
from scipy.optimize import minimize
from keras.utils.generic_utils import get_custom_objects
from util import one_hot_encode_aa, partition_data, get_balaji_predictions, get_samples, get_argmax
from util import convert_idx_array_to_aas, build_pred_vae_model, get_experimental_X_y
from util import get_gfp_X_y_aa
from losses import neg_log_likelihood
import json
import tensorflow_probability as tfp

tfd = tfp.distributions

Using TensorFlow backend.


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def killoran_opt(X_train, vae, oracles, ground_truth,
                 steps=10000, epsilon1=10**-5, epsilon2=1, noise_std=10**-5,
                 LD=100, verbose=False, adam=False):
    L = X_train.shape[1]
    
    G = vae.decoder_
    f = oracles
    
    sess = K.get_session()
    zt = K.tf.Variable(np.random.normal(size=[1, LD]), dtype='float32')
    pred_input = K.tf.Variable(np.zeros((1, L, X_train.shape[2])), dtype='float32')
    gen_output = G(zt)
    prior = tfd.Normal(0, 1)
    p_z = prior.log_prob(zt)
    predictions = K.tf.reduce_mean([f[i](pred_input)[0, 0] for i in range(len(f))])
    update_pred_input = K.tf.assign(pred_input, gen_output)
    dfdx = K.tf.gradients(ys=-predictions, xs=pred_input)[0]
    dfdz = K.tf.gradients(gen_output, zt, grad_ys=dfdx)[0]
    dpz = K.tf.gradients(p_z, zt)[0]
    
    noise = K.tf.random_normal(shape=[1, LD], stddev=noise_std)
    eps1 = K.tf.Variable(epsilon1, trainable=False)
    eps2 = K.tf.Variable(epsilon2, trainable=False)
    if adam:
        optimizer = K.tf.train.AdamOptimizer(learning_rate=epsilon2)
        step = dfdz + noise
    else:
        optimizer = K.tf.train.GradientDescentOptimizer(learning_rate=1)
        step = eps1 * dpz + eps2 * dfdz + noise
    
    design_op = optimizer.apply_gradients([(step, zt)])
    adam_initializers = [var.initializer for var in K.tf.global_variables() if 'Adam' in var.name or 'beta' in var.name]
    sess.run(adam_initializers)
    sess.run(pred_input.initializer)
    sess.run(zt.initializer)
    sess.run(eps1.initializer)
    sess.run(eps2.initializer)

    s = sess.run(K.tf.shape(zt))
    sess.run(update_pred_input, {zt: np.random.normal(size=s)})
    z_0 = sess.run([zt])
    results = np.zeros((steps, 2))
    xt_prev = None
    for t in range(steps):
        xt0, _, = sess.run([gen_output, design_op], {eps1: epsilon1, eps2:epsilon2})
        pred_in, preds = sess.run([update_pred_input, predictions])
        xt = get_argmax(xt0)
        ft = get_balaji_predictions(oracles, xt)[0][0]
        xt_seq = np.argmax(xt, axis=-1)
        if xt_prev is None or not np.all(xt_seq == xt_prev):
            xt_prev = xt_seq
            gt = ground_truth.predict(xt_seq)[:, 0][0]
        else:
            gt = results[t-1, 1]
        results[t, 0] = ft
        results[t, 1] = gt
    return results, {}

In [4]:
def train_experimental_gans():
    TRAIN_SIZE = 5000
    train_size_str = "%ik" % (TRAIN_SIZE/1000)
    for it in range(1):  # Can only use the first oracle and training set....b/c it can't take an ensemble
        RANDOM_STATE = it + 1
        X_train, y_train, _  = get_experimental_X_y(random_state=RANDOM_STATE, train_size=TRAIN_SIZE)
        
        L = X_train.shape[1]
        LD=100
        gan = WGAN(input_shape=(L, X_train.shape[2],), latent_dim=LD, gumbel=False)

        gan.train(X_train,
                 batch_size=1000,
                 epochs=100,
                 verbose=2
                 ) 

        suffix = "_%s_%i" % (train_size_str, RANDOM_STATE)
        gan.generator.save_weights("models/killoran_gan_geneorator_weights%s.h5" % suffix)
        gan.critic.save_weights("models/killoran_gan_critic_weights%s.h5" % suffix)
        gan.combined.save_weights("models/killoran_gan_combined_weights%s.h5" % suffix)

In [5]:
def run_killoran(killoran=True):
    TRAIN_SIZE = 5000
    train_size_str = "%ik" % (TRAIN_SIZE/1000)
    for i in range(3):
        RANDOM_STATE = i+1
        print(RANDOM_STATE)
        num_models = [1, 5, 20][i]
        X_train, _, _  = get_experimental_X_y(random_state=RANDOM_STATE, train_size=TRAIN_SIZE)

        LD=20
        L = X_train.shape[1]
        
        vae_suffix = '_%s_%i' % (train_size_str, RANDOM_STATE)
        
        ground_truth = SequenceGP(load=True, load_prefix="data/gfp_gp")
        loss = neg_log_likelihood
        get_custom_objects().update({"neg_log_likelihood": loss})
        oracle_suffix = '_%s_%i_%i' % (train_size_str, num_models, RANDOM_STATE)
        
        
        sess = tf.Session(graph=tf.get_default_graph())
        K.set_session(sess)
        vae = build_vae(latent_dim=20,
                  n_tokens=20, 
                  seq_length=X_train.shape[1],
                  enc1_units=50)
        vae.encoder_.load_weights("models/vae_0_encoder_weights%s.h5" % vae_suffix)
        vae.decoder_.load_weights("models/vae_0_decoder_weights%s.h5"% vae_suffix)
        vae.vae_.load_weights("models/vae_0_vae_weights%s.h5"% vae_suffix)
        
        oracles = [load_model("models/oracle_%i%s.h5" % (i, oracle_suffix)) for i in range(num_models)]
        if not killoran:
            results, test_max = killoran_opt(X_train, vae, oracles, ground_truth,
                                   steps=30000, epsilon1=1e-5, epsilon2=1.,  
                                   noise_std=1e-5,
                                   LD=20, verbose=False, adam=False)
            
            np.save("results/mala_results_%s_%i.npy" % (train_size_str, RANDOM_STATE), results)
            suffix = "_%s_%i" % (train_size_str, RANDOM_STATE)
            with open('results/%s_max%s.json'% ('mala', suffix), 'w') as outfile:
                json.dump(test_max, outfile)
                
        else:
            results, test_max = killoran_opt(X_train, vae, oracles, ground_truth,
                                             steps=10000, epsilon1=0., epsilon2=0.1,  
                                             noise_std=1e-6,
                                             LD=20, verbose=False, adam=True)
            np.save("results/killoran_may_results_%s_%i.npy" % (train_size_str, RANDOM_STATE), results)
            suffix = "_%s_%i" % (train_size_str, RANDOM_STATE)
            with open('results/%s_max%s.json'% ('killoran', suffix), 'w') as outfile:
                json.dump(test_max, outfile)
    

In [6]:
# train_experimental_gans()
run_killoran(killoran=True)

1
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
2
3
